# Redes Neuronales 2021 
Integrantes de grupo:
- Crespo, Pilar
- Müller, Malena
- Scala, Tobías
## TP1: Sesgos en el dataset de SNLI

Uno de los datasets más famosos de Natural Language Inference es SNLI. En esta tarea se debe responder, dadas dos frases A y B, si B es implicación de A ("entailment"), B es contradictorio con A ("contradiction") o si lo que enuncia B es neutral respecto de A ("neutral"). Se dice que A es la premisa y B es la hipótesis.

En Gururangan et al., 2018 mostraron que este dataset tiene algunos sesgos, provocados por ejemplo por las heurísticas que tienen los humanos para generar estos pares de frases (A, B). Para ello, desarrollaron un modelo que aún sin observar la premisa A pudiera clasificar el par (A, B) en alguna de las tres clases del dataset.

En este trabajo práctico intentaremos predecir a qué clase pertenece cada una de las hipótesis sin observar la premisa.

# Importación de datos

In [2]:
# import nltk
# from nltk import data
# from nltk.tokenize import word_tokenize
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# from nltk.corpus import stopwords
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import json

# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('stopwords')
# lemmatizer = WordNetLemmatizer()
# stemmer = PorterStemmer()

Importamos el archivo original del dataset para el procesamiento de datos y nos quedamos con las labels y las segundas oraciones de cada caso, ya que son las hipótesis. La clasificación de a qué clase pertenece cada par de oraciones se hace a partir del análisis de la oración que es la hipótesis.

In [3]:
# file = pd.read_json('snli_1.0_train.jsonl', lines=True)
# fileClass = file['gold_label']
# fileLines = file['sentence2']
# #print(fileClass)

# Filtrado / procesamiento de datos 

Tenemos dos tipos de data set. En uno realizamos lo siguiente:
- word_tokenize: Separamos la oración en strings.
- isalpha: Se eliminan los números.
- lemmatize: Se pasa todo a singular y se generaliza el género.
- stem: Se pasan todos los verbos a infinitivo y se deja todo en minúscula.

Y el otro dataset lo obtenemos de la misma forma que el recién mencionado, pero agregándole también la eliminación de stopwords (palabras comunes).

Si bien inicialmente probamos eliminando las stopwords, determinamos que convenía no hacerlo ya que hay palabras comunes como "no", que serían eliminadas, pero en nuestro caso son necesarias estas palabras para determinar que una hipótesis corresponde a una "contradicción", por ejemplo. Para esto, nos basamos en el paper "Annotation Artifacts in Natural Language Inference Data".

In [4]:
# linesFilt = []
# for i in range(len(fileLines)):
#     if (i % 1000 == 0):
#         print(i)
#     tok=word_tokenize(fileLines[i]) #Separa la oración en strings.
#     alpha=[x for x in tok if x.isalpha()] #Saca palabras con números.
#     lem=[lemmatizer.lemmatize(x,pos='v') for x in alpha] #Pasa de plural a singular y generaliza el género.
#     #stop=[x for x in lem if x not in stopwords.words('english')] #Saca las palabras comunes.
#     stem=[stemmer.stem(x) for x in lem] #Verbos a infinitivo y pasa todo a minuscula.
#     linesFilt.append(" ".join(stem))

Guardamos en un json las hipotesis ya procesadas.

In [5]:
# with open('train_processed_.jsonl', 'w') as file:
#     for i in range(len(fileClass)):
#         data2jsonl = {'gold_label': fileClass[i], 'sentence2': linesFilt[i]}
#         json.dump(data2jsonl, file)
#         file.write('\n')

Repetimos lo anterior para los datasets de validación y de test.

In [6]:
# ---- Validation file:
# file = pd.read_json('snli_1.0_dev.jsonl', lines=True)
# fileClass = file['gold_label']
# fileLines = file['sentence2']
# print(fileClass)

# linesFilt = []
# for i in range(len(fileLines)):
#     if (i % 1000 == 0):
#         print(i)
#     tok=word_tokenize(fileLines[i]) #Separa la oración en strings.
#     alpha=[x for x in tok if x.isalpha()] #Saca palabras con números.
#     lem=[lemmatizer.lemmatize(x,pos='v') for x in alpha] #Pasa de plural a singular y generaliza el género.
#     #stop=[x for x in lem if x not in stopwords.words('english')] #Saca las palabras comunes.
#     stem=[stemmer.stem(x) for x in lem] #Verbos a infinitivo y pasa todo a minuscula.
#     linesFilt.append(" ".join(stem))

# with open('val_processed_.jsonl', 'w') as file:
#     for i in range(len(fileClass)):
#         data2jsonl = {'gold_label': fileClass[i], 'sentence2': linesFilt[i]}
#         json.dump(data2jsonl, file)
#         file.write('\n')

# ---- Test file:
# file = pd.read_json('snli_1.0_test.jsonl', lines=True)
# fileClass = file['gold_label']
# fileLines = file['sentence2']
# print(fileClass)

# linesFilt = []
# for i in range(len(fileLines)):
#     if (i % 1000 == 0):
#         print(i)
#     tok=word_tokenize(fileLines[i]) #Separa la oración en strings.
#     alpha=[x for x in tok if x.isalpha()] #Saca palabras con números.
#     lem=[lemmatizer.lemmatize(x,pos='v') for x in alpha] #Pasa de plural a singular y generaliza el género.
#     #stop=[x for x in lem if x not in stopwords.words('english')] #Saca las palabras comunes.
#     stem=[stemmer.stem(x) for x in lem] #Verbos a infinitivo y pasa todo a minuscula.
#     linesFilt.append(" ".join(stem))

# with open('test_processed_.jsonl', 'w') as file:
#     for i in range(len(fileClass)):
#         data2jsonl = {'gold_label': fileClass[i], 'sentence2': linesFilt[i]}
#         json.dump(data2jsonl, file)
#         file.write('\n')

# Entrenamiento de la red con Naive Bayes

Levantamos los datos ya procesados.

In [33]:
import numpy as np
import pandas as pd #Implementación de clasificador bayesiano.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

trainFile = pd.read_json('train_processed_.jsonl', lines=True)
trainClass = trainFile['gold_label'].tolist()
trainLines = trainFile['sentence2'].tolist()

valFile = pd.read_json('val_processed_.jsonl', lines=True)
valClass = valFile['gold_label'].tolist()
valLines = valFile['sentence2'].tolist()

Obtenemos las matrices dispersas que contienen la frecuencia (cantidad de ocurrencia) de cada palabra del vocabulario.

In [34]:
# countVect = CountVectorizer(max_df=0.8,min_df=10, ngram_range=(1,2)) #Recibe todos los artículos y arma los vectores de cuenta. Check "ngram_range"
tfidfVect = TfidfVectorizer(max_df=0.8,min_df=5, ngram_range=(1,2))#, max_features=1000) #As tf–idf is very often used for text features

# trainData = countVect.fit_transform(trainLines) #Learn a vocabulary dictionary of all tokens in the raw documents and return document-term matrix.
# print(trainData.shape) #En este sparse matrix se muestran las ocurrencias de cada palabra en cada linea.
trainData = tfidfVect.fit_transform(trainLines) #Learn vocabulary and idf from training set, return document-term matrix.
print(trainData.shape) #En este sparse matrix se muestran las ocurrencias de cada palabra en cada linea. Con tfidf las ocurrencias de las palabras no se representan con números enteros.

# valData = countVect.transform(valLines) #Transform documents to document-term matrix. Extract token counts out of raw text documents using the vocabulary fitted with fit.
# print(valData.shape)
valData = tfidfVect.transform(valLines) #Transform documents to document-term matrix. Extract token counts out of raw text documents using the vocabulary fitted with fit.
print(valData.shape)

(550152, 56323)
(10000, 56323)


Entrenamos el modelo utilizando Naive Bayes y obtenemos el score con los datos de train y el score con los nuevos datos (validation).

In [35]:
multiNB = MultinomialNB(alpha=0.8)
multiNB.fit(trainData, trainClass)

print(multiNB.score(trainData, trainClass))
print(multiNB.score(valData, valClass))

0.6645381639983132
0.6414


Comprobamos la efectividad del modelo prediciendo los nuevos datos.

In [36]:
from sklearn.metrics import precision_score
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

valPred = multiNB.predict(valData)

enc = LabelEncoder()
enc.fit(valClass)
valClass = enc.transform(valClass)
valClass = np_utils.to_categorical(valClass-1) #Convert integers to dummy variables (i.e. one hot encoded).
enc.fit(valPred)
valPred = enc.transform(valPred)
valPred = np_utils.to_categorical(valPred) #Convert integers to dummy variables (i.e. one hot encoded).

print(precision_score(valClass, valPred, average=None))
# print(precision_score(valClass, valPred, average='micro'))
# print(precision_score(valClass, valPred, average='macro'))
# print(precision_score(valClass, valPred, average='weighted'))

[0.63833581 0.63371105 0.67246377]


# Conclusiones de Multinomial Naive Bayes

...

# Entrenamiento de la red con MLP

Truncamos el vocabulario para no tener error por falta de memoria.

In [37]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD

trunSVD = TruncatedSVD(n_components=1000)
trainData = trunSVD.fit_transform(trainData)
valData = trunSVD.transform(valData)
print(trainData.shape)

# enc = LabelEncoder()
enc.fit(trainClass)
trainClass = enc.transform(trainClass)
trainClass = np_utils.to_categorical(trainClass-1) #Convert integers to dummy variables (i.e. one hot encoded).
#print(trainClass_)
# valClass = enc.transform(valClass)
# valClass = np_utils.to_categorical(valClass-1) #Convert integers to dummy variables (i.e. one hot encoded).

(550152, 1000)


Generamos la estructura de la red neuronal con la siguiente función.

In [38]:
def neuralNetwork():
    Nwords = trainData.shape[1] #cantidad de datos (palabras del vocabulario). ###
    model = Sequential()
    model.add(Dense(200, input_shape=(Nwords,), activation='relu'))
    model.add(Dense(trainClass.shape[1], activation='softmax')) #La salida de la función softmax puede ser utilizada para representar una distribución categórica. 
                                                #Es empleada en varios métodos de clasificación multiclase tales como Regresión Logística Multinomial.
                                                #La función softmax es utilizada como capa final de los clasificadores basados en redes neuronales.
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Entrenamos la red neuronal y obtenemos el accuracyy del mismo.

In [39]:
#Cross validation.
# estClass = KerasClassifier(build_fn=neuralNetwork, epochs=50, batch_size=256, verbose=0) #Estimator.
# Kfold = KFold(n_splits=10, shuffle=True)
# scores = cross_val_score(estClass, trainData_.todense(), encClass_, cv=Kfold)
# print("Score: %.2f%% (%.2f%%)" % (scores.mean()*100, scores.std()*100))
#Hold out
model = neuralNetwork()
model.fit(trainData, trainClass, epochs=10, batch_size=256, verbose=1)
loss, acc = model.evaluate(valData, valClass)
print('Test Accuracy: %f' % (acc*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 200)               200200    
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 603       
Total params: 200,803
Trainable params: 200,803
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2150/2150 [==============================] - 9s 4ms/step - loss: 0.8663 - accuracy: 0.6032
Epoch 2/10
2150/2150 [==============================] - 10s 5ms/step - loss: 0.8338 - accuracy: 0.6216
Epoch 3/10
2150/2150 [==============================] - 9s 4ms/step - loss: 0.8131 - accuracy: 0.6354
Epoch 4/10
2150/2150 [==============================] - 8s 4ms/step - loss: 0.7946 - accuracy: 0.6466
Epoch 5/10
2150/2150 [==============================] - 7s 3ms/step - loss: 0.7790 - accuracy: 0.6555

Comprobamos la eficiencia del modelo con los nuevos datos (validation).

In [40]:
valPred = model.predict(valData, verbose = 1).round()

print(precision_score(valClass, valPred, average=None))
# print(precision_score(valClass_, valPred, average='micro'))
# print(precision_score(valClass_, valPred, average='macro'))
# print(precision_score(valClass_, valPred, average='weighted'))

313/313 [==============================] - 0s 528us/step
[0.71124744 0.67965217 0.68814815]


# Conclusiones de MLP

...